<div class="alert alert-info">
    <center><b>Set up Notebook</b></center>
</div>

In [ ]:
%pip install pandas tqdm pathlib

In [ ]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from utils.postgres import initialize_db
from utils.git import download
from utils.worker import get_optimal_max_workers

<div class="alert alert-info">
    <center><b>Create schema, database and tables</b></center>
</div>

In [ ]:
initialize_db()

<div class="alert alert-info">
    <center><b>Get repositories data from csv file</b></center>
</div>

In [ ]:
repos = pd.read_csv('../code_samples.csv', skiprows=1)
repos = repos.dropna(subset=['html_url'])

<div class="alert alert-info">
    <center><b>Cloning repositories in a subfolder</b></center>
</div>

In [ ]:
def download_wrapper(repo_row):
    repo_ecosystem = repo_row['html_url'].split('/')[-2]
    repo_name = repo_row['name']
    sample_name = f"{repo_ecosystem}/{repo_name}"
    download(sample_name)

In [ ]:
max_workers = get_optimal_max_workers()
with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i in range(len(repos)):
            repo = repos.iloc[i]
            futures.append(executor.submit(download_wrapper, repo))

        for _ in tqdm(as_completed(futures), total=len(futures), desc="Downloading Repositories"):
            pass